In [5]:
!pip install sklearn

  Using cached sklearn-0.0.post1.tar.gz (3.6 kB)
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2343 sha256=a3d098548908faf2ac457861c14b9c13783d48e22c4f61de604ba519e2ff539d
  Stored in directory: c:\users\ikenn\appdata\local\pip\cache\wheels\14\25\f7\1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built sklearn


In [38]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn import svm

import xgboost as xgb

import pickle

In [7]:
urldata = pd.read_csv('urldata_nlp.csv')

In [9]:
urldata = urldata.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)

In [11]:
def type_encoder(x):
    if x == 'malicious':
        return 1
    elif x == 'benign':
        return 0
    else:
        return x

urldata['target'] = urldata['type'].apply(lambda x: type_encoder(x))

urldata[['type', 'target']]

,type,target
0,malicious,1
1,benign,0
2,benign,0
3,malicious,1
4,malicious,1
...,...,...
1090222,malicious,1
1090223,malicious,1
1090224,malicious,1
1090225,malicious,1


In [12]:
num_features = ['https', 'digits', 'letters','url_len','len_subdomain','is_subdomain', 'ratioUrlDomLen',  'count_spl', 
                'ratioSpltinurl','Shortining_Service', 'having_ip_address', '@', '?', '-', '=', '.', '#', '%', '+', '$', '!', '*', ',', '//']
cat_features = ['tld']

In [13]:
enc = OneHotEncoder(handle_unknown='ignore')
scaler = MinMaxScaler()

In [14]:
#Creating a pipeline transformer for encoding categorical attributes and normalising numerical attributes
pipeline = ColumnTransformer([
("num", scaler, num_features),
("cat", enc, cat_features)])

In [41]:
filename = 'pipeline_model.sav'
pickle.dump(pipeline, open(filename, 'wb'))

In [23]:
X = pd.read_csv('urldata.csv').drop(['Unnamed: 0'], axis=1)
y = urldata['target']

In [24]:
X.head()

,url_len,@,?,-,=,.,#,%,+,$,...,digits,letters,Shortining_Service,having_ip_address,tld,len_subdomain,is_subdomain,ratioUrlDomLen,count_spl,ratioSpltinurl
0,16,0,0,1,0,2,0,0,0,0,...,0,13,0,0,com.br,0,0,0.000015,3,0.187500
1,35,0,0,0,0,2,0,0,0,0,...,1,29,0,0,com,0,0,0.000032,2,0.057143
2,31,0,0,0,0,2,0,0,0,0,...,1,25,0,0,org,0,0,0.000028,2,0.064516
3,88,0,1,1,4,2,0,0,0,0,...,7,60,0,0,be,0,0,0.000081,9,0.102273
4,235,0,1,1,3,2,0,0,0,0,...,22,199,0,0,net,0,0,0.000216,8,0.034043


In [25]:
X_trans = pipeline.fit_transform(X)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X_trans,y,test_size=0.2,shuffle=True)


In [27]:
adaB_lexical = AdaBoostClassifier()
adaB_lexical.fit(X_train,y_train)
y_pred = adaB_lexical.predict(X_test)
y_prob = adaB_lexical.predict_proba(X_test)[:,1]
print(classification_report(y_test,y_pred))
print('Confusion Matrix:',confusion_matrix(y_test, y_pred))
 
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

              precision    recall  f1-score   support

           0       0.90      0.98      0.94    154732
           1       0.93      0.73      0.82     63314

    accuracy                           0.91    218046
   macro avg       0.92      0.85      0.88    218046
weighted avg       0.91      0.91      0.90    218046

Confusion Matrix: [[151411   3321]
 [ 17115  46199]]
AUC: 0.938612920110172


In [28]:
filename = 'adaB_lexical_model.sav'
pickle.dump(adaB_lexical, open(filename, 'wb'))

In [29]:
xgb_lexical= xgb.XGBClassifier(n_estimators= 1000)
xgb_lexical.fit(X_train,y_train)
y_pred = xgb_lexical.predict(X_test)
y_prob = xgb_lexical.predict_proba(X_test)[:,1]
print(classification_report(y_test,y_pred))
print('Confusion Matrix:',confusion_matrix(y_test, y_pred))
 
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

              precision    recall  f1-score   support

           0       0.97      0.99      0.98    154732
           1       0.97      0.94      0.95     63314

    accuracy                           0.97    218046
   macro avg       0.97      0.96      0.97    218046
weighted avg       0.97      0.97      0.97    218046

Confusion Matrix: [[153036   1696]
 [  3928  59386]]
AUC: 0.9932557599970762


In [30]:
filename = 'xgB_lexical_model.sav'
pickle.dump(xgb_lexical, open(filename, 'wb'))

In [35]:
from sklearn.naive_bayes import GaussianNB
gnb_lexical= GaussianNB()
gnb_lexical.fit(X_train.toarray(),y_train)
y_pred = gnb_lexical.predict(X_test.toarray())
y_prob = gnb_lexical.predict_proba(X_test.toarray())[:,1]
print(classification_report(y_test,y_pred))
print('Confusion Matrix:',confusion_matrix(y_test, y_pred))
 
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

              precision    recall  f1-score   support

           0       0.74      0.99      0.85    154732
           1       0.90      0.14      0.24     63314

    accuracy                           0.75    218046
   macro avg       0.82      0.57      0.54    218046
weighted avg       0.79      0.75      0.67    218046

Confusion Matrix: [[153798    934]
 [ 54653   8661]]
AUC: 0.5666566052669362


In [36]:
rf_lexical =  RandomForestClassifier(n_estimators=100)
rf_lexical.fit(X_train,y_train)
y_pred = rf_lexical.predict(X_test)
y_prob = rf_lexical.predict_proba(X_test)[:,1]
print(classification_report(y_test,y_pred))
print('Confusion Matrix:',confusion_matrix(y_test, y_pred))
 
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

              precision    recall  f1-score   support

           0       0.97      0.99      0.98    154732
           1       0.97      0.94      0.95     63314

    accuracy                           0.97    218046
   macro avg       0.97      0.96      0.97    218046
weighted avg       0.97      0.97      0.97    218046

Confusion Matrix: [[152713   2019]
 [  3983  59331]]
AUC: 0.9912904458741468


In [37]:
filename = 'rf_lexical_model.sav'
pickle.dump(rf_lexical, open(filename, 'wb'))

In [39]:
svm_lexical = svm.SVC(probability=True)
svm_lexical.fit(X_train, y_train)

#Predict y value for test dataset
y_predict = svm_lexical.predict(X_test)
y_prob = svm_lexical.predict_proba(X_test)[:,1]
print(classification_report(y_test,y_predict))
print('Confusion Matrix:',confusion_matrix(y_test, y_predict))
 
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)


              precision    recall  f1-score   support

           0       0.91      0.98      0.94    154732
           1       0.95      0.75      0.84     63314

    accuracy                           0.92    218046
   macro avg       0.93      0.87      0.89    218046
weighted avg       0.92      0.92      0.91    218046

Confusion Matrix: [[152083   2649]
 [ 15527  47787]]
AUC: 0.9450875039531976


In [40]:
filename = 'svm_lexical_model.sav'
pickle.dump(svm_lexical, open(filename, 'wb'))